# Navigation Project

## Introduction

The aim of this project is to train an agent in a square world full of bananas, each yellow banana gives a reward of +1 point whereas each blue banana gives a reawrd of -1. An example of the banana world can be seen here:

<img src="banana_collector.gif" width="300" align="center">

The goal of each agent is to collect as many yellow bananas as possible while avoiding the blue bananas.

The state space has 37 dimensions, while the agent has 4 available actions:

- Up
- Down
- Right
- Left

The goal is to solve the environment, in order to solve it, the agent must average a score greater than 13 over 100 consecutive episodes.

## Environment

In [1]:
!pip -q install ./python

In [2]:
from unityagents import UnityEnvironment
import numpy as np

# Setting the environment with the default brain
env = UnityEnvironment(file_name="data/Banana_Windows_x86_64/Banana.x86_64")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [3]:
brain_name = env.brain_names[0]
brain = env.brains[brain_name]